## SRJ

# Aer simulator - qiskit device

1. Use cpq - office pc

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore")
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))

        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=2) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668944
Total excitations are 26
The adapt iteration now is 0
Highest gradient excitation is [2, 3, 6, 7]
Final updated parameters: [1.12299241]
Final cost: -1.430083670840328
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(1.12)─┤  State
3: ──X─├FermionicDoubleExcitation(1.12)─┤  State
6: ────├FermionicDoubleExcitation(1.12)─┤  State
7: ────╰FermionicDoubleExcitation(1.12)─┤  State
The adapt iteration now is 1
Highest gradient excitation is [0, 3, 5, 6]
Final updated parameters: [1.09716154 1.37775771]
Final cost: -1.5694603835520364
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(1.38)─┤  State
1: ──X──────────────────────────────────├FermionicDoubleExcitation(1.38)─┤  State
2: ──X─╭FermionicDoubleExcitation(1.10)─├FermionicDoubleExcitation(1.38)─┤  State
3: ──X─├FermionicDoubleExci

In [2]:
params, ash_excitation

(array([1.09716154, 1.37775771]), [[2, 3, 6, 7], [0, 3, 5, 6]])

In [3]:
qubits
hf_state = np.array([1,1,1,1,0,0,0,0])
len(ash_excitation)

2

## Conversion of pennylane to qasm code

In [4]:
from functools import partial
dev = qml.device('default.qubit', wires = 8)

#@partial(qml.set_shots, shots=100)
@qml.qnode(dev)
def circ(params, ash_excitation):
 [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
 for i, excitations in enumerate(ash_excitation):
    #print('The exc. going in are', ash_excitation)
    #print('Params', params)
    if len(ash_excitation[i]) == 4:
        qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
        #qml.DoubleExcitation(params[i], wires=ash_excitation[i])
    elif len(ash_excitation[i]) == 2:
        #qml.SingleExcitation(params[i], wires=ash_excitation[i])
        qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
 return qml.sample()

#circ(params, ash_excitation)
    
print(qml.to_openqasm(circ)(params, ash_excitation))

qc = (qml.to_openqasm(circ)(params, ash_excitation))


OPENQASM 2.0;
include "qelib1.inc";
qreg q[7];
creg c[7];
x q[0];
x q[1];
x q[2];
x q[3];
h q[2];
h q[3];
rx(-1.5707963267948966) q[4];
h q[5];
cx q[2],q[3];
cx q[3],q[4];
cx q[4],q[5];
rz(0.1371451924899199) q[5];
cx q[4],q[5];
cx q[3],q[4];
cx q[2],q[3];
h q[2];
h q[3];
rx(1.5707963267948966) q[4];
h q[5];
rx(-1.5707963267948966) q[2];
h q[3];
rx(-1.5707963267948966) q[4];
rx(-1.5707963267948966) q[5];
cx q[2],q[3];
cx q[3],q[4];
cx q[4],q[5];
rz(0.1371451924899199) q[5];
cx q[4],q[5];
cx q[3],q[4];
cx q[2],q[3];
rx(1.5707963267948966) q[2];
h q[3];
rx(1.5707963267948966) q[4];
rx(1.5707963267948966) q[5];
h q[2];
rx(-1.5707963267948966) q[3];
rx(-1.5707963267948966) q[4];
rx(-1.5707963267948966) q[5];
cx q[2],q[3];
cx q[3],q[4];
cx q[4],q[5];
rz(0.1371451924899199) q[5];
cx q[4],q[5];
cx q[3],q[4];
cx q[2],q[3];
h q[2];
rx(1.5707963267948966) q[3];
rx(1.5707963267948966) q[4];
rx(1.5707963267948966) q[5];
h q[2];
h q[3];
h q[4];
rx(-1.5707963267948966) q[5];
cx q[2],q[3];
cx q[3],q[

## Conversion to qiskit - qasm to qiskit & Measuring no of gates

In [5]:
import qiskit.qasm2

qcircuit = qiskit.qasm2.loads(qc)  #qc = qasm file
print(qcircuit)
#qcircuit.draw('mpl') #Printing the circuit

#Counting no of CNOTs

op_counts = qcircuit.count_ops()

# Print the counts
print(op_counts)

# Access the count of CNOT gates specifically
cnot_count = op_counts.get('cx', 0)
print(f"Number of CNOT gates: {cnot_count}")


        ┌───┐    ┌───┐                                                    »
q_0: ───┤ X ├────┤ H ├──■─────────────────────────────────────────────────»
        ├───┤    └───┘┌─┴─┐                                               »
q_1: ───┤ X ├─────────┤ X ├───────────────────────────────────────────────»
        ├───┤    ┌───┐└───┘                                               »
q_2: ───┤ X ├────┤ H ├──■──────────────────────────────────────────■──────»
        ├───┤    ├───┤┌─┴─┐                                      ┌─┴─┐    »
q_3: ───┤ X ├────┤ H ├┤ X ├──■─────────────────────────────■─────┤ X ├────»
     ┌──┴───┴───┐└───┘└───┘┌─┴─┐                         ┌─┴─┐┌──┴───┴──┐ »
q_4: ┤ Rx(-π/2) ├──────────┤ X ├──■───────────────────■──┤ X ├┤ Rx(π/2) ├─»
     └──┬───┬───┘          └───┘┌─┴─┐┌─────────────┐┌─┴─┐├───┤├─────────┴┐»
q_5: ───┤ H ├───────────────────┤ X ├┤ Rz(0.13715) ├┤ X ├┤ H ├┤ Rx(-π/2) ├»
     ┌──┴───┴───┐               └───┘└─────────────┘└───┘└───┘└──────────┘»
q_6: ┤ Rx(-π

## Loading qasm into pennylane

In [6]:
loaded_circ = qml.from_qasm(qc)
print(qml.draw(loaded_circ)())

0: ──X──────────H─╭●────────────────────────────────────────────────────────────────────────── ···
1: ──X────────────╰X────────────────────────────────────────────────────────────────────────── ···
2: ──X──────────H─╭●───────────────────────╭●──────────H──────────RX(-1.57)─╭●──────────────── ···
3: ──X──────────H─╰X─╭●─────────────────╭●─╰X──────────H──────────H─────────╰X─╭●───────────── ···
4: ──RX(-1.57)───────╰X─╭●───────────╭●─╰X──RX(1.57)───RX(-1.57)───────────────╰X─╭●────────── ···
5: ──H──────────────────╰X──RZ(0.14)─╰X──H──RX(-1.57)─────────────────────────────╰X──RZ(0.14) ···
6: ──RX(-1.57)──────────────────────────────────────────────────────────────────────────────── ···

0: ··· ───────────────────────────────────────────────────────────────────────────────────────── ···
1: ··· ───────────────────────────────────────────────────────────────────────────────────────── ···
2: ··· ──────────────╭●──────────RX(1.57)───H─────────╭●──────────────────────────────╭●──────── ···
3: 

# To display the extended circuit

In [7]:
expanded_new_state = qml.transforms.decompose(circ)

print(qml.draw(expanded_new_state, max_length=200)(params, ash_excitation))

0: ──X──────────H─╭●──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── ···
1: ──X────────────╰X──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── ···
2: ──X──────────H─╭●───────────────────────╭●──────────H──────────RX(-1.57)─╭●──────────────────────────────╭●──────────RX(1.57)───H─────────╭●──────────────────────────────╭●──────────H─────────H ···
3: ──X──────────H─╰X─╭●─────────────────╭●─╰X──────────H──────────H─────────╰X─╭●─────────────────╭●────────╰X──────────H──────────RX(-1.57)─╰X─╭●─────────────────╭●────────╰X──────────RX(1.57)──H ···
5: ──────────────────│──────────────────│──────────────────────────────────────│──────────────────│─────────────────────────────────────────────│──────────────────│────────────────────────────────